In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from tqdm.notebook import tqdm
from collections import Counter

# local imports
from prepare import *
from evaluate import *

In [2]:
raw_train, raw_train_labels, raw_test, specs, sample = read_raw_csvs()

## Shorten dataframes to speed up testing

In [9]:
%%script false
## Comment out above line to run this cell

frames = [raw_train, raw_train_labels, raw_test]#, specs, sample]

for i, df in enumerate(frames):
    frames[i] = frames[i][0:round(0.1*frames[i].shape[0])]
    print(frames[i].shape)

Couldn't find program: 'false'


In [4]:
train = raw_train.copy()
train_labels = raw_train_labels.copy()
test = raw_test.copy()

In [5]:
train.shape

(11341042, 11)

## Run modified remove_dead_weight function

In [6]:
train = remove_dead_weight(train, train_labels)
test = remove_dead_weight(test, train_labels, test_set=True)

## Process time

In [7]:
# convert timestamp to correct datetime type
train['timestamp'] = pd.to_datetime(train['timestamp'], infer_datetime_format=True)
test['timestamp'] = pd.to_datetime(test['timestamp'], infer_datetime_format=True)

In [15]:
# have the ability to break datetime into year, month, day, etc with add_datepart in library

## Encode data

https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63

In [8]:
train, test = encode_col(train, test, 'title')
train, test = encode_col(train, test, 'world')

In [95]:
#train.info(null_counts=True,verbose=True)

In [96]:
train.shape

(7696400, 59)

## Bin event codes

In [97]:
#TBD

## Process Data

https://www.kaggle.com/ragnar123/truncated-train-ensemble

In [98]:
# compiled_train_data = process_data(train)
# compiled_test_data = process_data(test, test_set=True)

# len(compiled_train_data)

# import joblib
# joblib.dump(compiled_train_data, 'compiled_train_data.pkl')
# joblib.dump(compiled_test_data, 'compiled_test_data.pkl')


['compiled_test_data.pkl']

In [18]:
compiled_train_data = joblib.load('compiled_train_data.pkl')
compiled_test_data = joblib.load('compiled_test_data.pkl')

compiled_train = pd.concat(compiled_train_data, axis=1).T

compiled_train.drop('installation_id_slice', axis=1, inplace=True)

compiled_test = pd.concat(compiled_test_data, axis=1).T

In [20]:
#assert len(set(sample.installation_id).difference(set(test.installation_id))) == 0
#compiled_train.shape, compiled_test.shape

## Add accuracy group from train_labels dataframe

In [21]:
# just verifying
train_labels.columns

Index(['game_session', 'installation_id', 'title', 'num_correct',
       'num_incorrect', 'accuracy', 'accuracy_group'],
      dtype='object')

In [22]:
compiled_train = pd.merge(compiled_train, train_labels[['installation_id','game_session','accuracy_group']], \
                  on=['installation_id','game_session'])
compiled_train.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,...,avg_event_count,avg_review_incorrect_feedback,avg_review_correct_feedback,total_rounds_beat,total_movies_skipped,total_movies_watched,total_elsewhere_clicks,total_help_button_clicks,total_play_again,accuracy_group
0,3bfd1a65,901acc108f55a5a1,2019-08-06 05:22:01.344000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,...,56.2855,313.143,4015.89,18,1,2,94,4,0,3
1,f56e0afc,77b8ee947eb84b4e,2019-08-06 05:35:19.167000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Bird Measurer (Assessment),Assessment,...,67.083,1771.27,3103.88,23,1,2,156,4,0,0
2,3bfd1a65,6bdf9623adc94d89,2019-08-06 05:37:50.020000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,...,65.4517,1871.73,3103.88,23,1,2,160,4,0,3
3,3bfd1a65,9501794defd84e4d,2019-08-06 20:34:53.812000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,...,62.609,2427.52,2544.59,47,2,5,348,4,1,2
4,f56e0afc,a9ef3ecb3d1acc6a,2019-08-06 20:49:59.095000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Bird Measurer (Assessment),Assessment,...,62.1572,2467.7,2309.92,58,2,5,387,4,1,3


## Convert as many columns as possible to numeric

In [23]:
def numerize(df):
    for i, column in enumerate(df.columns):
        col = df.columns[i]
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df

In [24]:
# for i, column in enumerate(compiled_train.columns):
#     col = compiled_train.columns[i]
#     compiled_train[col] = pd.to_numeric(compiled_train[col], errors='ignore')

# compiled_train.info()

compiled_train = numerize(compiled_train)
compiled_test = numerize(compiled_test)


In [25]:
compiled_train.shape, compiled_test.shape

((17690, 70), (1000, 69))

## Train RF model

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [27]:
#scaler = StandardScaler()

# get numeric fields only
#X = compiled_train._get_numeric_data().drop('accuracy_group', axis=1).iloc[:,:-3]

X = compiled_train.drop('accuracy_group', axis=1)._get_numeric_data()
print(X.shape)
#print(compiled_test.shape)
#X_scaled = scaler.fit_transform(X)
y = compiled_train.accuracy_group

compiled_test = compiled_test._get_numeric_data()
print(compiled_test.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

(17690, 62)
(1000, 62)


C:\Users\604572\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.4943470887507066

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier, 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model.SGDClassifier

In [ ]:
logreg = LogisticRegression()
quick_acc(compiled_train, logreg)

In [33]:
from evaluate import quick_acc

In [22]:
from sklearn.neighbors import KNeighborsClassifier

In [70]:
test_prediction = rf.predict(compiled_test)
list(zip(compiled_test.installation_id, test_prediction))

# To create a submission:
submission = pd.DataFrame()
submission['installation_id'] = compiled_test.installation_id
submission['accuracy_group'] = test_prediction
submission.head()

submission.accuracy_group.value_counts()

3    666
0    223
1     62
2     49
Name: accuracy_group, dtype: int64

In [71]:
submission.to_csv('submission_unscaled_1_1_2020.csv',index=False)

## Balance classes

### Balancing the classes prior to training seems to drop accuracy substantially

In [72]:
%%script false

from sklearn.utils import resample

# Separate classes
df_0 = compiled_train[compiled_train.accuracy_group==0]
df_1 = compiled_train[compiled_train.accuracy_group==1]
df_2 = compiled_train[compiled_train.accuracy_group==2]
df_3 = compiled_train[compiled_train.accuracy_group==3]

# Downsample 3, 0, 1 to 2's level - n=419
resampled_dfs = [df_2]

for i in [df_0,df_1,df_3]:  
    downsampled_df = resample(i, 
                              replace=False, # sample without replacement
                              n_samples = min(compiled_train.accuracy_group.value_counts()), #  to match minority
                              random_state = 42)  # reproducibility
    resampled_dfs.append(downsampled_df)

balanced_compiled_train = pd.concat(resampled_dfs, axis=0)

balanced_compiled_train.accuracy_group.value_counts()

3    2205
2    2205
1    2205
0    2205
Name: accuracy_group, dtype: int64

In [80]:
%%script false
scaler = StandardScaler()

X = balanced_compiled_train._get_numeric_data().drop('accuracy_group', axis=1)
X_scaled = scaler.fit_transform(X)
y = balanced_compiled_train.accuracy_group

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=.2, random_state=42)
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

Couldn't find program: 'false'


In [81]:
%%script false
test_prediction = rf.predict(compiled_test_scaled)
list(zip(compiled_test.installation_id, test_prediction))

Couldn't find program: 'false'


In [82]:
%script false
submission = pd.DataFrame()
submission['installation_id'] = compiled_test.installation_id
submission['accuracy_group'] = test_prediction
submission.head()

UsageError: Line magic function `%script` not found (But cell magic `%%script` exists, did you mean that instead?).


In [78]:
%%script false
submission.accuracy_group.value_counts()

submission.to_csv('submission_01022020.csv',index=False)

0    560
1    215
2    132
3     93
Name: accuracy_group, dtype: int64